In [1]:
import numpy as np

# =========================
# TSP + Metropolis（NumPyのみ最小）
#  - 状態：巡回路 perm（都市の並び）
#  - 提案：2-opt（区間反転）
#  - 受理：min(1, exp(-(ΔL)/T))
#  - ΔL は 2-opt で O(1) 計算（全距離を毎回足さない）
# =========================

def dist_matrix(xy: np.ndarray) -> np.ndarray:
    d = xy[:, None, :] - xy[None, :, :]
    return np.sqrt((d * d).sum(axis=2))

def tour_length(perm: np.ndarray, D: np.ndarray) -> float:
    nxt = np.roll(perm, -1)
    return float(D[perm, nxt].sum())

def delta_len_2opt(perm: np.ndarray, i: int, k: int, D: np.ndarray) -> float:
    """
    2-opt: perm[i:k+1] を反転する場合の距離差 ΔL を O(1) で返す
    i < k、ただし「隣接辺」や「全反転で無意味」になりやすいケースは呼び出し側で避ける
    """
    n = perm.size
    a = perm[i - 1]          # i の直前
    b = perm[i]
    c = perm[k]
    d = perm[(k + 1) % n]    # k の直後（巡回）

    # 旧: (a-b) + (c-d)
    # 新: (a-c) + (b-d)
    return (D[a, c] + D[b, d]) - (D[a, b] + D[c, d])

def metropolis_tsp(
    xy: np.ndarray,
    steps: int = 200_000,
    T: float = 1.0,
    seed: int = 0,
    report_every: int = 20_000
):
    rng = np.random.default_rng(seed)
    D = dist_matrix(xy)
    n = xy.shape[0]

    perm = rng.permutation(n)
    curL = tour_length(perm, D)

    best_perm = perm.copy()
    bestL = curL

    for t in range(1, steps + 1):
        # 2-opt の区間 [i, k] を選ぶ（i < k）
        i, k = rng.integers(0, n, size=2)
        if i == k:
            continue
        if i > k:
            i, k = k, i

        # つまらない/壊れやすいケースを避ける（最小のガード）
        # - 区間が全体（0..n-1）だと同値
        # - 隣接（k=i+1）も改善が弱いことが多いので避ける（なくても動く）
        if (i == 0 and k == n - 1) or (k == i + 1):
            continue

        dL = delta_len_2opt(perm, i, k, D)

        # 受理判定（dL<=0 は必ず受理、悪化は確率で受理）
        if dL <= 0.0 or rng.random() < np.exp(-dL / T):
            # 2-opt 実行（区間反転）
            perm[i:k+1] = perm[i:k+1][::-1]
            curL += dL

            if curL < bestL:
                bestL = curL
                best_perm = perm.copy()

        if report_every and (t % report_every == 0):
            print(f"step={t}  現在={curL:.3f}  最良={bestL:.3f}")

    return best_perm, bestL

if __name__ == "__main__":
    # 例：ランダム都市（平面上）
    n = 60
    rng = np.random.default_rng(1)
    xy = rng.random((n, 2))  # [0,1]×[0,1]

    best_perm, bestL = metropolis_tsp(xy, steps=200_000, T=0.02, seed=0, report_every=50_000)
    print("最良距離:", bestL)
    print("最良ルート(先頭20都市):", best_perm[:20])


step=50000  現在=6.486  最良=6.359
step=100000  現在=6.427  最良=6.299
step=150000  現在=6.306  最良=6.212
step=200000  現在=6.393  最良=6.212
最良距離: 6.212036133880737
最良ルート(先頭20都市): [11  0 20 25 16 31 23 49  6 56 26 14  1 24 48 33 35 18  8 47]
